一、数据集准备
    1、下载LUNA16数据集到本地，放在data文件夹中
    2、进入路径./data/LUNA16, 将得到的subset0-9.rar和seg-lungs-LUNA16.rar共11个文件解压
    3、设置路径，运行下面代码，划分训练集和验证集，并且将数据格式转化为niffi，得到如下的文件结构
    ./train
        /image
        /seg
    ./val
        /image
        /seg
    注：
        （1）LUNA16数据集共887个volume，我们选择subset9的最后三个volume作为验证集，subset0-8进而subset9剩下的部分作为训练集。
        （2）可以通过下载ITK-SNAP软件可视化图片和分割标签

In [ ]:
# Function: Adjust Data Content Structure & Convert volume to nifti format
# Author: Zhang Zhongzhou
# Date: 2022/9/16
# LUNA16
#   /train
#       /image
#       /seg
#   /val
#       /image
#       /seg
import os
import os.path as osp
import shutil
import SimpleITK as sitk
import glob
from tqdm import tqdm
from src.model_utils.config import config

def convert_nifti(input_im, save_image, roi_size):
    """
    Convert dataset into mifti format.

    Args:
        input_im(str): input image name
        save_image(str): output image name
        roi_size(list): The size to crop the image
    """
    img = sitk.ReadImage(input_im)
    image_array = sitk.GetArrayFromImage(img)
    D, H, W = image_array.shape
    if H < roi_size[0] or W < roi_size[1] or D < roi_size[2]:
        print("file {} size is smaller than roi size, ignore it.".format(input_im))
        # continue
    sitk.WriteImage(img, save_image)


if __name__ == "__main__":

    root = "../data/LUNA16"
    image_fold = ["subset0","subset1","subset2","subset3","subset4","subset5","subset6","subset7","subset8","subset9"]
    label_fold = "seg-lungs-LUNA16"

    save_train_image_path = osp.join(root, "train", "image")
    save_train_seg_path = osp.join(root, "train", "seg")
    save_val_image_path = osp.join(root, "val", "image")
    save_val_seg_path = osp.join(root, "val", "seg")

    # create save path
    if not os.path.exists(save_train_image_path):
        os.makedirs(save_train_image_path)
    if not os.path.exists(save_train_seg_path):
        os.makedirs(save_train_seg_path)
    if not os.path.exists(save_val_image_path):
        os.makedirs(save_val_image_path)
    if not os.path.exists(save_val_seg_path):
        os.makedirs(save_val_seg_path)

    # split train and val
    # Here we select the last 10 volume in subset9 for validation
    for subset in image_fold:
        image_list = sorted(glob.glob(osp.join(root, subset, "*.mhd")))
        for im in tqdm(image_list):
                im_name = im.split("/")[-1][:-4]
                seg = osp.join(root, label_fold, im_name + ".mhd")
                save_im_name = osp.join(save_train_image_path, im_name+".nii.gz")
                save_seg_name = osp.join(save_train_seg_path, im_name + ".nii.gz")
                convert_nifti(im, save_im_name, config.roi_size)
                convert_nifti(seg, save_seg_name, config.roi_size)

    all_images = sorted(os.listdir(save_train_image_path))[-10:]
    all_segs = sorted(os.listdir(save_train_seg_path))[-10:]
    for i, name in enumerate(all_images):
        sou_im = osp.join(save_train_image_path, name)
        sou_seg = osp.join(save_train_seg_path, name)
        shutil.move(sou_im, save_val_image_path)
        shutil.move(sou_seg, save_val_seg_path)

In [ ]:
# visualize on ITK-SNAP
import cv2
im = cv2.imread("image/visual_data.jpg", 1)
cv2.imshow("vis_data", im)
cv2.waitKey(0)

二、参数设置（训练参数和测试参数）

In [3]:
import ml_collections
import warnings
warnings.filterwarnings("ignore")

def get_config():
    """
    Get Config according to the yaml file and cli arguments.
    """
    cfg = ml_collections.ConfigDict()
    # Builtin Configurations(DO NOT CHANGE THESE CONFIGURATIONS unless you know exactly what you are doing)
    # args = argparse.ArgumentParser(description="default name", add_help=False)
    cfg.enable_fp16_gpu=False
    cfg.enable_modelarts=False
    # Url for modelarts
    cfg.data_url=""
    cfg.train_ur=""
    cfg.checkpoint_url=""
    # Path for local
    cfg.run_distribute=False
    cfg.enable_profiling=False
    cfg.data_path="data/LUNA16/train/"
    cfg.output_path="output"
    cfg.load_path="/checkpoint_path/"
    cfg.device_target="GPU"
    cfg.checkpoint_path="./checkpoint/"
    cfg.checkpoint_file_path="Unet3d-10-110.ckpt"

    # ==============================================================================
    # data loader options
    cfg.num_worker = 4
    # Training options
    cfg.max_epoch = 10
    cfg.lr=0.0005
    cfg.batch_size=1
    cfg.epoch_size=10
    cfg.warmup_step=120
    cfg.warmup_ratio=0.3
    cfg.num_classes=4
    cfg.in_channels=1
    cfg.keep_checkpoint_max=1
    cfg.loss_scale=256.0
    cfg.roi_size=[224, 224, 96]
    cfg.overlap=0.25
    cfg.min_val=-500
    cfg.max_val=1000
    cfg.upper_limit=5
    cfg.lower_limit=3

    # Export options
    cfg.device_id=0
    cfg.ckpt_file="./checkpoint/Unet3d-10-110.ckpt"
    cfg.file_name="unet3d"
    cfg.file_format="MINDIR"

    # 310 infer options
    cfg.pre_result_path="./preprocess_Result"
    cfg.post_result_path="./result_Files"

    return cfg

config = get_config()
# print(config)

三、定义数据预处理的transform

In [6]:
# transforms
import re
import numpy as np
import logger

np_str_obj_array_pattern = re.compile(r'[SaUO]')

def correct_nifti_head(img):
    """
    Check nifti object header's format, update the header if needed.
    In the updated image pixdim matches the affine.

    Args:
        img: nifti image object
    """
    dim = img.header["dim"][0]
    if dim >= 5:
        return img
    pixdim = np.asarray(img.header.get_zooms())[:dim]
    norm_affine = np.sqrt(np.sum(np.square(img.affine[:dim, :dim]), 0))
    if np.allclose(pixdim, norm_affine):
        return img
    # if hasattr(img, "get_sform"):
    #     return rectify_header_sform_qform(img)
    return img

class LoadData:
    """
    Load Image data from provided files.
    """
    def __init__(self, canonical=False, dtype=np.float32):
        """
        Args:
        canonical: if True, load the image as closest to canonical axis format.
        dtype: convert the loaded image to this data type.
        """
        self.canonical = canonical
        self.dtype = dtype

    def operation(self, filename):
        img_array = list()
        compatible_meta = dict()
        filename = filename.item()
        filename = [filename]
        for name in filename:
            img = nib.load(str(name)[2:-1])
            img = correct_nifti_head(img)
            header = dict(img.header)
            header["filename_or_obj"] = name
            header["affine"] = img.affine
            header["original_affine"] = img.affine.copy()
            header["canonical"] = self.canonical
            ndim = img.header["dim"][0]
            spatial_rank = min(ndim, 3)
            header["spatial_shape"] = img.header["dim"][1 : spatial_rank + 1]
            if self.canonical:
                img = nib.as_closest_canonical(img)
                header["affine"] = img.affine
            img_array.append(np.array(img.get_fdata(dtype=self.dtype)))
            img.uncache()
            if not compatible_meta:
                for meta_key in header:
                    meta_datum = header[meta_key]
                    if isinstance(meta_datum, np.ndarray) \
                        and np_str_obj_array_pattern.search(meta_datum.dtype.str) is not None:
                        continue
                    compatible_meta[meta_key] = meta_datum
            else:
                assert np.allclose(header["affine"], compatible_meta["affine"])

        img_array = np.stack(img_array, axis=0) if len(img_array) > 1 else img_array[0]
        return img_array

    def __call__(self, img, label):
        img_array = self.operation(img)
        seg_array = self.operation(label)
        return img_array, seg_array

class ExpandChannel:
    """
    Expand a 1-length channel dimension to the input image.
    """
    def __call__(self, img, label):
        img_array = img[None]
        seg_array = label[None]
        return img_array, seg_array

class Orientation:
    """
    Change the input image's orientation into the specified based on `ax`.
    """
    def __init__(self, ax="RAS", labels=tuple(zip("LPI", "RAS"))):
        """
        Args:
            ax: N elements sequence for ND input's orientation.
            labels: optional, None or sequence of (2,) sequences
                (2,) sequences are labels for (beginning, end) of output axis.
        """
        self.ax = ax
        self.labels = labels

    def operation(self, data, affine=None):
        """
        original orientation of `data` is defined by `affine`.

        Args:
            data: in shape (num_channels, H[, W, ...]).
            affine (matrix): (N+1)x(N+1) original affine matrix for spatially ND `data`. Defaults to identity.

        Returns:
            data (reoriented in `self.ax`), original ax, current ax.
        """
        if data.ndim <= 1:
            raise ValueError("data must have at least one spatial dimension.")
        if affine is None:
            affine = np.eye(data.ndim, dtype=np.float64)
            affine_copy = affine
        else:
            affine_copy = to_affine_nd(data.ndim-1, affine)
        src = nib.io_orientation(affine_copy)
        dst = nib.orientations.axcodes2ornt(self.ax[:data.ndim-1], labels=self.labels)
        spatial_ornt = nib.orientations.ornt_transform(src, dst)
        ornt = spatial_ornt.copy()
        ornt[:, 0] += 1
        ornt = np.concatenate([np.array([[0, 1]]), ornt])
        data = nib.orientations.apply_orientation(data, ornt)
        return data

    def __call__(self, img, label):
        img_array = self.operation(img)
        seg_array = self.operation(label)
        return img_array, seg_array

class ScaleIntensityRange:
    """
    Apply specific intensity scaling to the whole numpy array.
    Scaling from [src_min, src_max] to [tgt_min, tgt_max] with clip option.

    Args:
        src_min: intensity original range min.
        src_max: intensity original range max.
        tgt_min: intensity target range min.
        tgt_max: intensity target range max.
        is_clip: whether to clip after scaling.
    """
    def __init__(self, src_min, src_max, tgt_min, tgt_max, is_clip=False):
        self.src_min = src_min
        self.src_max = src_max
        self.tgt_min = tgt_min
        self.tgt_max = tgt_max
        self.is_clip = is_clip

    def operation(self, data):
        if self.src_max - self.src_min == 0.0:
            logger.warning("Divide by zero (src_min == src_max)")
            return data - self.src_min + self.tgt_min
        data = (data - self.src_min) / (self.src_max - self.src_min)
        data = data * (self.tgt_max - self.tgt_min) + self.tgt_min
        if self.is_clip:
            data = np.clip(data, self.tgt_min, self.tgt_max)
        return data

    def __call__(self, image, label):
        image = self.operation(image)
        return image, label

class RandomCropSamples:
    def __init__(self, roi_size, num_samples=1):
        self.roi_size = roi_size
        self.num_samples = num_samples
        self.set_random_state(0)

    def set_random_state(self, seed=None):
        """
        Set the random seed to control the slice size.

        Args:
            seed: set the random state with an integer seed.
        """
        MAX_SEED = np.iinfo(np.uint32).max + 1
        if seed is not None:
            _seed = seed % MAX_SEED
            self.rand_fn = np.random.RandomState(_seed)
        else:
            self.rand_fn = np.random.RandomState()
        return self

    def get_random_patch(self, dims, patch_size, rand_fn=None):
        """
        Returns a tuple of slices to define a random patch in an array of shape `dims` with size `patch_size`.
        """
        rand_int = np.random.randint if rand_fn is None else rand_fn.randint
        min_corner = tuple(rand_int(0, ms - ps + 1) if ms > ps else 0 for ms, ps in zip(dims, patch_size))
        return tuple(slice(mc, mc + ps) for mc, ps in zip(min_corner, patch_size))

    def get_random_slice(self, img_size):
        slices = (slice(None),) + self.get_random_patch(img_size, self.roi_size, self.rand_fn)
        return slices

    def __call__(self, image, label):
        res_image = []
        res_label = []
        for _ in range(self.num_samples):
            slices = self.get_random_slice(image.shape[1:])
            img = image[slices]
            label_crop = label[slices]
            res_image.append(img)
            res_label.append(label_crop)
        return np.array(res_image), np.array(res_label)

class OneHot:
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def one_hot(self, labels):
        N, K = labels.shape
        one_hot_encoding = np.zeros((N, self.num_classes, K), dtype=np.float32)
        for i in range(N):
            for j in range(K):
                one_hot_encoding[i, labels[i][j], j] = 1
        return one_hot_encoding

    def operation(self, labels):
        N, _, D, H, W = labels.shape
        labels = labels.astype(np.int32)
        labels = np.reshape(labels, (N, -1))
        labels = self.one_hot(labels)
        labels = np.reshape(labels, (N, self.num_classes, D, H, W))
        return labels

    def __call__(self, image, label):
        label = self.operation(label)
        return image, label

class ConvertLabel:
    """
    Crop at the center of image with specified ROI size.
    Args:
        roi_size: the spatial size of the crop region e.g. [224,224,128]
        If its components have non-positive values, the corresponding size of input image will be used.
    """
    def operation(self, data):
        """
        Apply the transform to `img`, assuming `img` is channel-first and
        slicing doesn't apply to the channel dim.
        """
        data[data > config.upper_limit] = 0
        data = data - (config.lower_limit - 1)
        data = np.clip(data, 0, config.lower_limit)
        return data

    def __call__(self, image, label):
        label = self.operation(label)
        return image, label


四、创建dataloader

In [7]:
import glob
import mindspore.dataset as ds
from mindspore.dataset.transforms.transforms import Compose
import nibabel as nib
import os

class Dataset:
    def __init__(self, data, seg):
        self.data = data
        self.seg = seg
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        data = self.data[index]
        seg = self.seg[index]
        return [data], [seg]

def create_dataset(data_path, seg_path, rank_size=1, rank_id=0, is_training=True):
    seg_files = sorted(glob.glob(os.path.join(seg_path, "*.nii.gz")))
    train_files = [os.path.join(data_path, os.path.basename(seg)) for seg in seg_files]
    train_ds = Dataset(data=train_files, seg=seg_files)
    train_loader = ds.GeneratorDataset(train_ds, column_names=["image", "seg"], num_parallel_workers=config.num_worker, \
                                       shuffle=is_training, num_shards=rank_size, shard_id=rank_id)

    if is_training:
        transform_image = Compose([LoadData(),
                                   ExpandChannel(),
                                   Orientation(),
                                   ScaleIntensityRange(src_min=config.min_val, src_max=config.max_val, tgt_min=0.0, \
                                                       tgt_max=1.0, is_clip=True),
                                   RandomCropSamples(roi_size=config.roi_size, num_samples=4),
                                   ConvertLabel(),
                                   OneHot(num_classes=config.num_classes)])
    else:
        transform_image = Compose([LoadData(),
                                   ExpandChannel(),
                                   Orientation(),
                                   ScaleIntensityRange(src_min=config.min_val, src_max=config.max_val, tgt_min=0.0, \
                                                       tgt_max=1.0, is_clip=True),
                                   ConvertLabel()])

    train_loader = train_loader.map(operations=transform_image,
                                    input_columns=["image", "seg"],
                                    num_parallel_workers=config.num_worker,
                                    python_multiprocessing=True)
    if not is_training:
        train_loader = train_loader.batch(1)
    return train_loader

if __name__=="__main__":
    # cfg.data_path
    train_dataset = create_dataset(data_path=config.data_path+"image",\
                                   seg_path=config.data_path+"seg", \
                                   is_training=True)
    train_data_size = train_dataset.get_dataset_size()
    print("create dataloader successfully!!")
    print("train dataset length is:", train_data_size)
    # for image, label in train_dataset.create_tuple_iterator():
    #     print("image size: {}".format(image.shape))
    #     break

create dataloader successfully!!
train dataset length is: 877


五、构建UNet3d网络结构

In [8]:
import mindspore as ms
import mindspore.nn as nn
from mindspore import dtype as mstype
from mindspore.ops import operations as P
from src.unet3d_parts import Down, Up
import numpy as np

class UNet3d_(nn.Cell):
    """
    UNet3d_ support fp32 and fp16(amp) training on GPU.
    """
    def __init__(self):
        super(UNet3d_, self).__init__()
        self.n_channels = config.in_channels
        self.n_classes = config.num_classes

        # down
        self.down1 = Down(in_channel=self.n_channels, out_channel=16, dtype=mstype.float32)
        self.down2 = Down(in_channel=16, out_channel=32, dtype=mstype.float32)
        self.down3 = Down(in_channel=32, out_channel=64, dtype=mstype.float32)
        self.down4 = Down(in_channel=64, out_channel=128, dtype=mstype.float32)
        self.down5 = Down(in_channel=128, out_channel=256, stride=1, kernel_size=(1, 1, 1), \
                          dtype=mstype.float32)
        # up
        self.up1 = Up(in_channel=256, down_in_channel=128, out_channel=64, \
                      dtype=mstype.float32)
        self.up2 = Up(in_channel=64, down_in_channel=64, out_channel=32, \
                      dtype=mstype.float32)
        self.up3 = Up(in_channel=32, down_in_channel=32, out_channel=16, \
                      dtype=mstype.float32)
        self.up4 = Up(in_channel=16, down_in_channel=16, out_channel=self.n_classes, \
                      dtype=mstype.float32, is_output=True)

    def construct(self, input_data):
        x1 = self.down1(input_data)
        x2 = self.down2(x1)
        x3 = self.down3(x2)
        x4 = self.down4(x3)
        x5 = self.down5(x4)

        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        return x

class UNet3d(nn.Cell):
    def __init__(self):
        super(UNet3d, self).__init__()
        self.n_channels = config.in_channels
        self.n_classes = config.num_classes

        # down
        self.transpose = P.Transpose()
        self.down1 = Down(in_channel=self.n_channels, out_channel=16, dtype=mstype.float16).to_float(mstype.float16)
        self.down2 = Down(in_channel=16, out_channel=32, dtype=mstype.float16).to_float(mstype.float16)
        self.down3 = Down(in_channel=32, out_channel=64, dtype=mstype.float16).to_float(mstype.float16)
        self.down4 = Down(in_channel=64, out_channel=128, dtype=mstype.float16).to_float(mstype.float16)
        self.down5 = Down(in_channel=128, out_channel=256, stride=1, kernel_size=(1, 1, 1), \
                          dtype=mstype.float16).to_float(mstype.float16)
        # up
        self.up1 = Up(in_channel=256, down_in_channel=128, out_channel=64, \
                      dtype=mstype.float16).to_float(mstype.float16)
        self.up2 = Up(in_channel=64, down_in_channel=64, out_channel=32, \
                      dtype=mstype.float16).to_float(mstype.float16)
        self.up3 = Up(in_channel=32, down_in_channel=32, out_channel=16, \
                      dtype=mstype.float16).to_float(mstype.float16)
        self.up4 = Up(in_channel=16, down_in_channel=16, out_channel=self.n_classes, \
                      dtype=mstype.float16, is_output=True).to_float(mstype.float16)

        self.cast = P.Cast()

    def construct(self, input_data):
        input_data = self.cast(input_data, mstype.float16)
        x1 = self.down1(input_data)
        x2 = self.down2(x1)
        x3 = self.down3(x2)
        x4 = self.down4(x3)
        x5 = self.down5(x4)

        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.cast(x, mstype.float32)
        return x

if __name__=="__main__":
    x = ms.Tensor(np.zeros([1, 1, 224, 224, 96]), ms.float32)
    model = UNet3d().set_train(True)
    out = model(x)

    # print(model)
    print("UNet3D model input:", x.shape)
    print("UNet3D model output:", out.shape)
    print(model)

UNet3D model input: (1, 1, 224, 224, 96)
UNet3D model output: (1, 4, 224, 224, 96)
UNet3d<
  (down1): Down<
    (down_conv): ResidualUnit<
      (down_conv_1): Conv3d<input_channels=1, output_channels=16, kernel_size=(3, 3, 3), stride=(2, 2, 2), pad_mode=pad, padding=1, dilation=(1, 1, 1), group=1, has_bias=False, weight_init=normal, bias_init=zeros, format=NCDHW>
      (batchNormal1): BatchNorm3d<
        (bn2d): BatchNorm2d<num_features=16, eps=1e-05, momentum=0.09999999999999998, gamma=Parameter (name=down1.down_conv.batchNormal1.bn2d.gamma, shape=(16,), dtype=Float32, requires_grad=True), beta=Parameter (name=down1.down_conv.batchNormal1.bn2d.beta, shape=(16,), dtype=Float32, requires_grad=True), moving_mean=Parameter (name=down1.down_conv.batchNormal1.bn2d.moving_mean, shape=(16,), dtype=Float32, requires_grad=False), moving_variance=Parameter (name=down1.down_conv.batchNormal1.bn2d.moving_variance, shape=(16,), dtype=Float32, requires_grad=False)>
        >
      (relu1): PReLU<>

[WARNING] KERNEL(10408,7f69f94a4080,python):2022-10-24-12:09:09.913.632 [mindspore/ccsrc/plugin/device/gpu/kernel/gpu_kernel_factory.cc:147] CheckSM] It is recommended to use devices with a computing capacity >= 7, but the current device's computing capacity is 6


六、自定义metrics

In [9]:
from medpy.metric import binary

class metrics:
    def __init__(self, smooth=1e-5):
        self.smooth=1e-5

    def dice_metric(self, y_pred, y_label, empty_score=1.0):
        """Calculates the dice coefficient for the images"""
        return binary.dc(y_pred, y_label)

    def jc_metric(self, y_pred, y_label):
        """Jaccard coefficient"""
        return binary.jc(y_pred, y_label)

    def hd95_metric(self, y_pred, y_label):
        """Calculates the hausdorff distance for the images"""
        return binary.hd95(y_pred, y_label, voxelspacing=1)

    def asd_metric(self, y_pred, y_label):
        """Average surface distance metric."""
        return binary.asd(y_pred, y_label, voxelspacing=None)

    def assd_metric(self, y_pred, y_label):
        """Average symmetric surface distance metric."""
        return binary.assd(y_pred, y_label, voxelspacing=None)

    def precision_metric(self, y_pred, y_label):
        """precision metric."""
        return binary.precision(y_pred, y_label, voxelspacing=None)

    def sensitivity_metric(self, y_pred, y_label, smooth = 1e-5):
        """recall(also sensitivity) metric."""
        return binary.recall(y_pred, y_label)

七、设置不同的学习率策略

In [10]:
# dyanmic learning rate
import math

def linear_warmup_learning_rate(current_step, warmup_steps, base_lr, init_lr):
    lr_inc = (float(base_lr) - float(init_lr)) / float(warmup_steps)
    learning_rate = float(init_lr) + lr_inc * current_step
    return learning_rate

def a_cosine_learning_rate(current_step, base_lr, warmup_steps, decay_steps):
    base = float(current_step - warmup_steps) / float(decay_steps)
    learning_rate = (1 + math.cos(base * math.pi)) / 2 * base_lr
    return learning_rate

def dynamic_lr(config, base_step):
    """dynamic learning rate generator"""
    base_lr = config.lr
    total_steps = int(base_step * config.epoch_size)
    warmup_steps = config.warmup_step
    lr = []
    for i in range(total_steps):
        if i < warmup_steps:
            lr.append(linear_warmup_learning_rate(i, warmup_steps, base_lr, base_lr * config.warmup_ratio))
        else:
            lr.append(a_cosine_learning_rate(i, base_lr, warmup_steps, total_steps))
    return lr

In [11]:
# Copyright 2021 Huawei Technologies Co., Ltd
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ============================================================================

"""Moxing adapter for ModelArts"""

import os
import functools
from mindspore import context
# from src.model_utils.config import config

_global_sync_count = 0

def get_device_id():
    device_id = os.getenv('DEVICE_ID', '0')
    return int(device_id)


def get_device_num():
    device_num = os.getenv('RANK_SIZE', '1')
    return int(device_num)


def get_rank_id():
    global_rank_id = os.getenv('RANK_ID', '0')
    return int(global_rank_id)


def get_job_id():
    job_id = os.getenv('JOB_ID')
    job_id = job_id if job_id != "" else "default"
    return job_id

def sync_data(from_path, to_path):
    """
    Download data from remote obs to local directory if the first url is remote url and the second one is local path
    Upload data from local directory to remote obs in contrast.
    """
    import moxing as mox
    import time
    global _global_sync_count
    sync_lock = "/tmp/copy_sync.lock" + str(_global_sync_count)
    _global_sync_count += 1

    # Each server contains 8 devices as most.
    if get_device_id() % min(get_device_num(), 8) == 0 and not os.path.exists(sync_lock):
        print("from path: ", from_path)
        print("to path: ", to_path)
        mox.file.copy_parallel(from_path, to_path)
        print("===finish data synchronization===")
        try:
            os.mknod(sync_lock)
        except IOError:
            pass
        print("===save flag===")

    while True:
        if os.path.exists(sync_lock):
            break
        time.sleep(1)

    print("Finish sync data from {} to {}.".format(from_path, to_path))


def moxing_wrapper(pre_process=None, post_process=None):
    """
    Moxing wrapper to download dataset and upload outputs.
    """
    def wrapper(run_func):
        @functools.wraps(run_func)
        def wrapped_func(*args, **kwargs):
            # Download data from data_url
            if config.enable_modelarts:
                if config.data_url:
                    sync_data(config.data_url, config.data_path)
                    print("Dataset downloaded: ", os.listdir(config.data_path))
                if config.checkpoint_url:
                    sync_data(config.checkpoint_url, config.load_path)
                    print("Preload downloaded: ", os.listdir(config.load_path))
                if config.train_url:
                    sync_data(config.train_url, config.output_path)
                    print("Workspace downloaded: ", os.listdir(config.output_path))

                context.set_context(save_graphs_path=os.path.join(config.output_path, str(get_rank_id())))
                config.device_num = get_device_num()
                config.device_id = get_device_id()
                if not os.path.exists(config.output_path):
                    os.makedirs(config.output_path)

                if pre_process:
                    pre_process()

            run_func(*args, **kwargs)

            # Upload data to train_url
            if config.enable_modelarts:
                if post_process:
                    post_process()

                if config.train_url:
                    print("Start to copy output directory")
                    sync_data(config.output_path, config.train_url)
        return wrapped_func
    return wrapper


八、主函数进行训练

In [ ]:
import os
import mindspore
import mindspore.nn as nn
from mindspore import ops
from mindspore import SummaryRecord
import mindspore.common.dtype as mstype
from mindspore import Tensor, Model, context
from mindspore.context import ParallelMode
from mindspore.communication.management import init, get_rank, get_group_size
from mindspore.train.loss_scale_manager import FixedLossScaleManager

if config.device_target == 'Ascend':
    device_id = int(os.getenv('DEVICE_ID'))
    context.set_context(mode=context.GRAPH_MODE, device_target=config.device_target, save_graphs=False, \
                        device_id=device_id)
else:
    context.set_context(mode=context.GRAPH_MODE, device_target=config.device_target, save_graphs=False)

mindspore.set_seed(1)

@moxing_wrapper()
def train_net(run_distribute=False):
    if run_distribute:
        init()
        if config.device_target == 'Ascend':
            rank_id = get_device_id()
            rank_size = get_device_num()
        else:
            rank_id = get_rank()
            rank_size = get_group_size()
        parallel_mode = ParallelMode.DATA_PARALLEL
        context.set_auto_parallel_context(parallel_mode=parallel_mode,
                                          device_num=rank_size,
                                          gradients_mean=True)
    else:
        rank_id = 0
        rank_size = 1
    # (1) create dataloader
    train_dataset = create_dataset(data_path=config.data_path + "/image/",
                                   seg_path=config.data_path + "/seg/",
                                   rank_size=rank_size,
                                   rank_id=rank_id, is_training=True)
    train_data_size = train_dataset.get_dataset_size()
    print("train dataset length is:", train_data_size)
    # (2) construct network
    if config.device_target == 'Ascend':
        network = UNet3d()
    else:
        network = UNet3d_()

    # (3) define loss funtion
    loss_ce_fn = nn.CrossEntropyLoss()
    loss_dice_fn = nn.DiceLoss(smooth=1e-5)
    # (4) lr shedule and optimizor
    lr = Tensor(dynamic_lr(config, train_data_size), mstype.float32)
    optimizer = nn.Adam(params=network.trainable_params(), learning_rate=lr)
    # (5) set training mode
    network.set_train()
    # (6) Start training
    print("============== Starting Training ==============")
    def forward_fn(data, label):
        logits = network(data)
        loss_ce = loss_ce_fn(logits, label)
        loss_dice = loss_dice_fn(logits, label)
        loss = loss_dice + loss_ce
        return loss, loss_dice, loss_ce, logits
    # Get gradient function
    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)
    # Define function of one-step training
    def train_step(data, label):
        (loss, loss_dice, loss_ce, _), grads = grad_fn(data, label)
        loss = ops.depend(loss, optimizer(grads))
        return loss, loss_dice, loss_ce

    with SummaryRecord('./summary_dir/summary_01') as summary_record:
        for epoch in range(config.max_epoch):
            for step, (data, label) in enumerate(train_dataset.create_tuple_iterator()):
                loss, loss_dice, loss_ce = train_step(data, label)

                current_step = epoch * train_data_size + step
                current_lr = optimizer.get_lr()
                summary_record.record(current_step)
                summary_record.add_value('scalar', 'lr', current_lr)
                summary_record.add_value('scalar', 'loss_total', loss)
                summary_record.add_value('scalar', 'loss_dice', loss_dice)
                summary_record.add_value('scalar', 'loss_ce', loss_ce)

                loss, loss_dice, loss_ce = loss.asnumpy(), loss_dice.asnumpy(), loss_ce.asnumpy()
                print("Epoch: %d [%d/%d] [%d/%d] lr:%.7f Loss: %.4f Loss_dice: %.4f Loss_ce: %.4f" %
                      (epoch, step, train_data_size, current_step, train_data_size*config.max_epoch,
                       current_lr, loss, loss_dice, loss_ce))
            # Save checkpoint
            ckpt_save_dir = os.path.join(config.output_path, config.checkpoint_path)
            mindspore.save_checkpoint(network, os.path.join(ckpt_save_dir, "Epoch_"+str(epoch)+"_model.ckpt"))
            print("Saved Model to {}/Epoch_{}_model.ckpt".format(ckpt_save_dir, epoch))

    print("============== End Training ==============")

if __name__ == '__main__':
    train_net()

九、模型预测

In [12]:
import math
import numpy as np

def first(iterable, default=None):
    """
    Returns the first item in the given iterable or `default` if empty, meaningful mostly with 'for' expressions.
    """
    for i in iterable:
        return i
    return default

def _get_scan_interval(image_size, roi_size, num_image_dims, overlap):
    """
    Compute scan interval according to the image size, roi size and overlap.
    Scan interval will be `int((1 - overlap) * roi_size)`, if interval is 0,
    use 1 instead to make sure sliding window works.
    """
    if len(image_size) != num_image_dims:
        raise ValueError("image different from spatial dims.")
    if len(roi_size) != num_image_dims:
        raise ValueError("roi size different from spatial dims.")

    scan_interval = []
    for i in range(num_image_dims):
        if roi_size[i] == image_size[i]:
            scan_interval.append(int(roi_size[i]))
        else:
            interval = int(roi_size[i] * (1 - overlap))
            scan_interval.append(interval if interval > 0 else 1)
    return tuple(scan_interval)

def dense_patch_slices(image_size, patch_size, scan_interval):
    """
    Enumerate all slices defining ND patches of size `patch_size` from an `image_size` input image.

    Args:
        image_size: dimensions of image to iterate over
        patch_size: size of patches to generate slices
        scan_interval: dense patch sampling interval

    Returns:
        a list of slice objects defining each patch
    """
    num_spatial_dims = len(image_size)
    patch_size = patch_size
    scan_num = []
    for i in range(num_spatial_dims):
        if scan_interval[i] == 0:
            scan_num.append(1)
        else:
            num = int(math.ceil(float(image_size[i]) / scan_interval[i]))
            scan_dim = first(d for d in range(num) if d * scan_interval[i] + patch_size[i] >= image_size[i])
            scan_num.append(scan_dim + 1 if scan_dim is not None else 1)
    starts = []
    for dim in range(num_spatial_dims):
        dim_starts = []
        for idx in range(scan_num[dim]):
            start_idx = idx * scan_interval[dim]
            start_idx -= max(start_idx + patch_size[dim] - image_size[dim], 0)
            dim_starts.append(start_idx)
        starts.append(dim_starts)
    out = np.asarray([x.flatten() for x in np.meshgrid(*starts, indexing="ij")]).T
    return [(slice(None),)*2 + tuple(slice(s, s + patch_size[d]) for d, s in enumerate(x)) for x in out]

def create_sliding_window(image, roi_size, overlap):
    num_image_dims = len(image.shape) - 2
    if overlap < 0 or overlap >= 1:
        raise AssertionError("overlap must be >= 0 and < 1.")
    image_size_temp = list(image.shape[2:])
    image_size = tuple(max(image_size_temp[i], roi_size[i]) for i in range(num_image_dims))

    scan_interval = _get_scan_interval(image_size, roi_size, num_image_dims, overlap)
    slices = dense_patch_slices(image_size, roi_size, scan_interval)
    windows_sliding = [image[slice] for slice in slices]
    return windows_sliding, slices

def CalculateDice(y_pred, label):
    """
    Args:
        y_pred: predictions. As for classification tasks,
            `y_pred` should has the shape [BN] where N is larger than 1. As for segmentation tasks,
            the shape should be [BNHW] or [BNHWD].
        label: ground truth, the first dim is batch.
    """
    metric = metrics()
    y_pred_output = np.expand_dims(np.argmax(y_pred, axis=1), axis=1)
    y_pred = one_hot(y_pred_output)
    y = one_hot(label)
    y_pred, y = ignore_background(y_pred, y)

    dice = metric.dice_metric(y_pred, y)
    hd95 = metric.hd95_metric(y_pred, y)
    jc = metric.jc_metric(y_pred, y)
    asd = metric.asd_metric(y_pred, y)
    sens = metric.sensitivity_metric(y_pred, y)
    return dice, hd95, jc, asd, sens

def ignore_background(y_pred, label):
    """
    This function is used to remove background (the first channel) for `y_pred` and `y`.
    Args:
        y_pred: predictions. As for classification tasks,
            `y_pred` should has the shape [BN] where N is larger than 1. As for segmentation tasks,
            the shape should be [BNHW] or [BNHWD].
        label: ground truth, the first dim is batch.
    """
    label = label[:, 1:] if label.shape[1] > 1 else label
    y_pred = y_pred[:, 1:] if y_pred.shape[1] > 1 else y_pred
    return y_pred, label

def one_hot(labels):
    N, _, D, H, W = labels.shape
    labels = np.reshape(labels, (N, -1))
    labels = labels.astype(np.int32)
    N, K = labels.shape
    one_hot_encoding = np.zeros((N, config.num_classes, K), dtype=np.float32)
    for i in range(N):
        for j in range(K):
            one_hot_encoding[i, labels[i][j], j] = 1
    labels = np.reshape(one_hot_encoding, (N, config.num_classes, D, H, W))
    return labels

In [13]:
import os
import numpy as np
from mindspore import dtype as mstype
from mindspore import Model, context, Tensor
from mindspore.train.serialization import load_checkpoint, load_param_into_net

config.device_target = "GPU"
if config.device_target == 'Ascend':
    device_id = int(os.getenv('DEVICE_ID'))
    context.set_context(mode=context.GRAPH_MODE, device_target=config.device_target, save_graphs=False, \
                        device_id=device_id)
else:
    context.set_context(mode=context.GRAPH_MODE, device_target=config.device_target, save_graphs=False)

# @moxing_wrapper()
def test_net(data_path, ckpt_path):
    data_dir = data_path + "/image/"
    seg_dir = data_path + "/seg/"
    eval_dataset = create_dataset(data_path=data_dir, seg_path=seg_dir, is_training=False)
    eval_data_size = eval_dataset.get_dataset_size()
    print("test dataset length is:", eval_data_size)

    metrics_score = {}
    metrics_score["dice"] = 0
    metrics_score["hd95"] = 0
    metrics_score["jc"] = 0

    if config.device_target == 'Ascend':
        network = UNet3d()
    else:
        network = UNet3d_()
    network.set_train(False)
    param_dict = load_checkpoint(ckpt_path)
    load_param_into_net(network, param_dict)
    # metrics
    results = {}
    results["dice"] = 0
    results["hd95"] = 0
    results["jc"] = 0
    results["asd"] = 0
    results["sens"] = 0
    model = Model(network)
    index = 0
    total_dice = 0
    config.batch_size=1
    for batch in eval_dataset.create_dict_iterator(num_epochs=1, output_numpy=True):
        image = batch["image"]
        seg = batch["seg"]
        print("current image shape is {}".format(image.shape), flush=True)
        sliding_window_list, slice_list = create_sliding_window(image, config.roi_size, config.overlap)
        image_size = (config.batch_size, config.num_classes) + image.shape[2:]
        output_image = np.zeros(image_size, np.float32)
        count_map = np.zeros(image_size, np.float32)
        importance_map = np.ones(config.roi_size, np.float32)
        for window, slice_ in zip(sliding_window_list, slice_list):
            window_image = Tensor(window, mstype.float32)
            pred_probs = model.predict(window_image)
            output_image[slice_] += pred_probs.asnumpy()
            count_map[slice_] += importance_map
        output_image = output_image / count_map
        dice, hd95, jc, asd, sens = CalculateDice(output_image, seg)

        print("The %d batch Dice: %.4f, HD95:%.4f, JC: %.4f, ASD: %.4f, Sens: %.4f"%(index, dice, hd95, jc, asd, sens))
        total_dice += dice
        results["dice"] += dice
        results["hd95"] += hd95
        results["jc"] += jc
        results["asd"] += asd
        results["sens"] += sens
        index = index + 1
    avg_dice = results["dice"] / eval_data_size
    avg_hd95 = results["hd95"] / eval_data_size
    avg_jc = results["jc"] / eval_data_size
    avg_asd = results["asd"] / eval_data_size
    avg_sens = results["sens"] / eval_data_size
    print("**********************End Eval***************************************")
    print("The average Dice: %.4f, HD95:%.4f, JC: %.4f, ASD: %.4f, Sens: %.4f" %
          (avg_dice, avg_hd95, avg_jc, avg_asd, avg_sens))

if __name__ == '__main__':
    test_net(data_path="data/LUNA16/val/",
             ckpt_path="./output/checkpoint/Epoch_9_model.ckpt")

test dataset length is: 10


[WARNING] ME(12961:140093130686592,_MPWorker-22):2022-10-24-12:09:42.356.231 [mindspore/dataset/engine/queue.py:120] Using shared memory queue, but rowsize is larger than allocated memory max_rowsize 16777216 current rowsize 119537664


current image shape is (1, 1, 512, 512, 140)
The 0 batch Dice: 0.9806, HD95:0.0000, JC: 0.9620, ASD: 0.0592, Sens: 0.9808
current image shape is (1, 1, 512, 512, 158)
The 1 batch Dice: 0.9847, HD95:0.0000, JC: 0.9698, ASD: 0.1013, Sens: 0.9817
current image shape is (1, 1, 512, 512, 133)
The 2 batch Dice: 0.9771, HD95:0.0000, JC: 0.9552, ASD: 0.5383, Sens: 0.9791
current image shape is (1, 1, 512, 512, 114)
The 3 batch Dice: 0.9581, HD95:0.0000, JC: 0.9196, ASD: 2.2986, Sens: 0.9883
current image shape is (1, 1, 512, 512, 350)
The 4 batch Dice: 0.9821, HD95:0.0000, JC: 0.9649, ASD: 0.0508, Sens: 0.9826
current image shape is (1, 1, 512, 512, 290)
The 5 batch Dice: 0.9753, HD95:0.0000, JC: 0.9517, ASD: 0.6219, Sens: 0.9759
current image shape is (1, 1, 512, 512, 172)
The 6 batch Dice: 0.9894, HD95:0.0000, JC: 0.9789, ASD: 0.0175, Sens: 0.9922
current image shape is (1, 1, 512, 512, 125)
The 7 batch Dice: 0.9585, HD95:0.0000, JC: 0.9203, ASD: 0.0504, Sens: 0.9576
current image shape is (